# Zenith WASM Plugin Testing

**Author:** Wahyu Ardiansyah  
**Repository:** [github.com/vibeswithkk/Zenith-dataplane](https://github.com/vibeswithkk/Zenith-dataplane)

---

This notebook demonstrates Zenith's WASM plugin system:

1. **Build WASM Plugin** - Compile Rust to WebAssembly
2. **Load Plugin** - Load into Zenith engine
3. **Test Filtering** - Process events through plugin
4. **Performance** - Measure plugin overhead

**Runtime:** ~10 minutes

## 1. Environment Setup

In [ ]:
%%time
# Install Rust toolchain
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y 2>/dev/null

import os
os.environ['PATH'] = f"{os.environ['HOME']}/.cargo/bin:{os.environ['PATH']}"

# Add WASM target (wasip1 for Rust 1.78+, fallback to wasi for older)
!rustup target add wasm32-wasip1 2>/dev/null || rustup target add wasm32-wasi 2>/dev/null || echo 'WASM target added'

# Verify
!rustc --version
!rustup target list --installed | grep wasm

In [ ]:
# Install Python dependencies
!pip install -q pyarrow pandas numpy

In [ ]:
# Clone Zenith repository
!rm -rf Zenith-dataplane
!git clone --depth 1 https://github.com/vibeswithkk/Zenith-dataplane.git

import os
os.chdir('Zenith-dataplane')
!pwd

## 2. Build Zenith Engine

In [ ]:
%%time
# Build zenith-core (includes WASM host)
!cargo build --release -p zenith-core 2>&1 | tail -20
print("\nzenith-core built successfully!")

## 3. Build WASM Plugin

In [ ]:
# View the filter plugin source code
!cat examples/plugins/filter_plugin.rs

In [ ]:
%%time
# Build the WASM plugin (try wasip1 first for newer Rust, fallback to wasi)
os.chdir('examples/plugins')

# Determine which target is available
import subprocess
result = subprocess.run(['rustup', 'target', 'list', '--installed'], capture_output=True, text=True)
if 'wasm32-wasip1' in result.stdout:
    target = 'wasm32-wasip1'
else:
    target = 'wasm32-wasi'

print(f"Using WASM target: {target}")
!cargo build --target {target} --release 2>&1 | tail -15

# Check output
!find . -name "*.wasm" -type f 2>/dev/null

os.chdir('../..')

In [ ]:
# Copy WASM file to examples directory
import shutil
import glob

# Try both wasip1 and wasi paths
wasm_paths = glob.glob('examples/plugins/target/wasm32-*/release/*.wasm')
wasm_dst = 'examples/custom_filter.wasm'

if wasm_paths:
    wasm_src = wasm_paths[0]
    shutil.copy(wasm_src, wasm_dst)
    size = os.path.getsize(wasm_dst)
    print(f"WASM plugin created: {wasm_dst}")
    print(f"Source: {wasm_src}")
    print(f"Size: {size} bytes")
else:
    # Use existing filter.wasm
    wasm_dst = 'examples/filter.wasm'
    if os.path.exists(wasm_dst):
        size = os.path.getsize(wasm_dst)
        print(f"Using existing plugin: {wasm_dst}")
        print(f"Size: {size} bytes")
    else:
        print("No WASM plugin found!")

## 4. Test WASM Host Directly

In [ ]:
# Run WASM-related tests
!cargo test -p zenith-core wasm_host --lib 2>&1 | tail -30

In [ ]:
# Run engine plugin tests
!cargo test -p zenith-core engine::tests::test_engine_load_plugin --lib 2>&1 | tail -20

## 5. WASM Binary Analysis

In [ ]:
# Install wasm analysis tools
!pip install -q wasmtime

import wasmtime

# Load and analyze WASM module
wasm_path = wasm_dst

print(f"Analyzing: {wasm_path}")
print("="*50)

try:
    # Read WASM bytes
    with open(wasm_path, 'rb') as f:
        wasm_bytes = f.read()
    
    # Create engine and module
    engine = wasmtime.Engine()
    module = wasmtime.Module(engine, wasm_bytes)
    
    # List exports
    print("\nExported Functions:")
    for export in module.exports:
        if export.type.func_type:
            params = list(export.type.func_type.params)
            results = list(export.type.func_type.results)
            print(f"  {export.name}({params}) -> {results}")
    
    print(f"\nModule size: {len(wasm_bytes)} bytes")
    print("WASM module is valid!")
    
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Call WASM function directly
import wasmtime

print("Testing WASM function calls")
print("="*50)

try:
    # Setup wasmtime
    config = wasmtime.Config()
    engine = wasmtime.Engine(config)
    store = wasmtime.Store(engine)
    
    # Load module
    with open(wasm_path, 'rb') as f:
        wasm_bytes = f.read()
    module = wasmtime.Module(engine, wasm_bytes)
    
    # Create linker and instantiate
    linker = wasmtime.Linker(engine)
    instance = linker.instantiate(store, module)
    
    # Get and call on_event function
    on_event = instance.exports(store).get("on_event")
    
    if on_event:
        print("\nTesting on_event(source_id, seq_no):")
        print("-"*40)
        
        test_cases = [
            (1, 1, "Normal event"),
            (1, 50, "Normal event"),
            (1, 100, "Seq divisible by 100"),
            (0, 1, "Reserved source_id=0"),
            (2, 200, "Seq divisible by 100"),
            (5, 999, "Normal event"),
        ]
        
        for source_id, seq_no, desc in test_cases:
            result = on_event(store, source_id, seq_no)
            status = "ALLOW" if result == 1 else "BLOCK"
            print(f"  on_event({source_id}, {seq_no:3d}) = {result} [{status}] - {desc}")
    
    # Call version if available
    version = instance.exports(store).get("version")
    if version:
        v = version(store)
        print(f"\nPlugin version: {v}")
    
    # Call init if available
    init = instance.exports(store).get("init")
    if init:
        result = init(store)
        print(f"Plugin init: {result}")
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

## 6. WASM Performance Test

In [ ]:
import time
import numpy as np

print("WASM Performance Benchmark")
print("="*50)

try:
    # Setup
    engine = wasmtime.Engine()
    store = wasmtime.Store(engine)
    
    with open(wasm_path, 'rb') as f:
        module = wasmtime.Module(engine, f.read())
    
    linker = wasmtime.Linker(engine)
    instance = linker.instantiate(store, module)
    on_event = instance.exports(store).get("on_event")
    
    if on_event:
        # Warmup
        for i in range(1000):
            on_event(store, 1, i)
        
        # Benchmark
        iterations = [1000, 10000, 100000]
        
        print(f"\n{'Calls':>10} {'Time (ms)':>12} {'Calls/sec':>15}")
        print("-"*40)
        
        for n in iterations:
            start = time.perf_counter()
            for i in range(n):
                on_event(store, 1, i)
            elapsed = time.perf_counter() - start
            
            calls_per_sec = n / elapsed
            print(f"{n:>10,} {elapsed*1000:>12.2f} {calls_per_sec:>15,.0f}")
        
        # Final throughput
        n = 100000
        times = []
        for _ in range(5):
            start = time.perf_counter()
            for i in range(n):
                on_event(store, 1, i)
            times.append(time.perf_counter() - start)
        
        avg_time = np.mean(times)
        throughput = n / avg_time
        
        print("\n" + "="*40)
        print(f"Average throughput: {throughput:,.0f} calls/sec")
        print(f"Latency per call: {avg_time/n*1e6:.2f} µs")
        
except Exception as e:
    print(f"Error: {e}")

## 7. Zenith Engine Integration Test

In [ ]:
# Run all WASM-related tests
!cargo test -p zenith-core --lib 2>&1 | grep -E "(test|passed|failed|ok|FAILED)"

## 8. Summary

In [ ]:
print("="*60)
print("ZENITH WASM PLUGIN TEST SUMMARY")
print("="*60)

print("\n[1] WASM Plugin System")
print("    Host: wasmtime (Rust)")
print(f"    Target: {target if 'target' in dir() else 'wasm32-wasip1'}")

print("\n[2] Plugin Interface")
print("    on_event(source_id: i32, seq_no: i64) -> i32")
print("    Returns: 1=allow, 0=block")

print("\n[3] Available Plugins")
print(f"    - examples/filter.wasm (built-in)")
if os.path.exists('examples/custom_filter.wasm'):
    print(f"    - examples/custom_filter.wasm (compiled)")

print("\n[4] Use Cases")
print("    - Event filtering (allow/block)")
print("    - Rate limiting")
print("    - Data validation")
print("    - Custom preprocessing")

print("\n" + "="*60)
print("WASM PLUGIN TESTING COMPLETE!")
print("="*60)

print("\nRepository: https://github.com/vibeswithkk/Zenith-dataplane")